In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time

Strategy: 
- Code to collect movie title and other features into nested list
- Code to automate capture of data from multiple web pages and store in dataframe   

In [2]:
def get_info_v2(soup):
    
    # nested list of films. contains movie title and corresponding features
    movies_list = []
    
    specs = soup.find_all('div',{'class':'lister-item-content'})


    for content in specs:
        
        # contains the information identified in the variables - features of project
        movie_attributes = []

        movie_year = content.find('span',class_='lister-item-year text-muted unbold').text.replace("(", "").replace(")", "") 
        movie_title = content.find_all('a')[0].text

        # movie deets contains many features. Below we get them indiviudally 
        movie_deets = content.find_all('p')[0]
        movie_runtime = movie_deets.find('span', class_='runtime').text
        movie_genre = movie_deets.find('span', class_='genre').text.replace('\n','').strip()
        movie_rating = movie_deets.find('span', class_ = 'certificate').text
        director_stars = content.find_all('p')[2]
        

        # director and gross box office $
        #final_gross = float(votes_gross_split[:-2])
        director = director_stars.find('a').text
        blob = director_stars.findNext('a')
        actor1 = blob.findNext('a').text
        
        votes_gross = content.find('p', class_= "sort-num_votes-visible").text
        votes_gross_split = votes_gross.split('$')[1]
        final_gross = float(votes_gross_split[:-2])
        
        # places variables into movie attribute list
        movie_attributes.append(movie_title)
        movie_attributes.append(movie_runtime)
        movie_attributes.append(movie_genre)
        movie_attributes.append(movie_rating)
        movie_attributes.append(final_gross)
        movie_attributes.append(director)
        movie_attributes.append(actor1)
        movie_attributes.append(movie_year)
        
        # append movie attributes to list 
        movies_list.append(movie_attributes)

    return movies_list

In [3]:
start_pages = [351,451,501,551,601,651,701]

#instantiates empty dataframe
master_df = pd.DataFrame()

master_list = []
for page in start_pages:
    all_url = 'https://www.imdb.com/search/title/?genres=thriller&sort=boxoffice_gross_us,desc&start=' + str(page) +'&explore=title_type,genres&ref_=adv_nxt'
    response = requests.get(all_url)
    page_html = response.text  
    soup = bs(page_html)
    curr_data = get_info_v2(soup)
    curr_movies = pd.DataFrame(curr_data, columns = ['Title', 'Duration', 'Genre','Rating', 'Gross','Director','Lead', 'Year']) 
    
    #concatonates each page into the empty dataframe
    master_df = pd.concat([master_df,curr_movies], axis=0)

In [4]:
master_df

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year
0,The Happening,91 min,"Adventure, Sci-Fi, Thriller",R,64.51,M. Night Shyamalan,Mark Wahlberg,2008
1,The Purge,85 min,"Horror, Sci-Fi, Thriller",R,64.47,James DeMonaco,Ethan Hawke,I 2013
2,Deja Vu,126 min,"Action, Crime, Sci-Fi",PG-13,64.04,Tony Scott,Denzel Washington,2006
3,Unknown,113 min,"Action, Mystery, Thriller",PG-13,63.69,Jaume Collet-Serra,Liam Neeson,I 2011
4,Thunderball,130 min,"Action, Adventure, Thriller",PG,63.60,Terence Young,Sean Connery,1965
...,...,...,...,...,...,...,...,...
45,Breach,110 min,"Biography, Crime, Drama",PG-13,33.23,Billy Ray,Chris Cooper,2007
46,Striptease,115 min,"Comedy, Crime, Drama",R,33.11,Andrew Bergman,Demi Moore,1996
47,Zodiac,157 min,"Crime, Drama, Mystery",R,33.08,David Fincher,Jake Gyllenhaal,2007
48,The Rite,114 min,"Drama, Horror, Mystery",PG-13,33.05,Mikael Håfström,Colin O'Donoghue,2011


In [6]:
#master_df.to_csv('thrillers_351_750rows.csv', index=False)